In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [2]:
trainDF = pd.read_csv('train.csv')
trainDF.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [3]:
testDF = pd.read_csv('test.csv')
testDF.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


In [4]:
concatDF = pd.concat([trainDF,testDF])
concatDF.loc[concatDF['Gender']=='Male',['Gender']]=0
concatDF.loc[concatDF['Gender']=='Female',['Gender']]=1
concatDF['Age']=concatDF['Age']/concatDF['Age'].max()
concatDF.loc[concatDF['Vehicle_Age']=='< 1 Year',['Vehicle_Age']]=0
concatDF.loc[concatDF['Vehicle_Age']=='1-2 Year',['Vehicle_Age']]=0.5
concatDF.loc[concatDF['Vehicle_Age']=='> 2 Years',['Vehicle_Age']]=1
concatDF['Annual_Premium']=concatDF['Annual_Premium']/concatDF['Annual_Premium'].max()
concatDF['Vintage']=concatDF['Vintage']/concatDF['Vintage'].max()
concatDF.loc[concatDF['Vehicle_Damage']=='No',['Vehicle_Damage']]=0
concatDF.loc[concatDF['Vehicle_Damage']=='Yes',['Vehicle_Damage']]=1
print(concatDF.info())
concatDF.head()


<class 'pandas.core.frame.DataFrame'>
Index: 19174664 entries, 0 to 7669865
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   float64
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               float64
 11  Response              float64
dtypes: float64(6), int64(3), object(3)
memory usage: 1.9+ GB
None


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,0,0.247059,1,35.0,0,0.5,1,0.120521,124.0,0.625418,0.0
1,1,0,0.505882,1,28.0,0,1,1,0.109061,26.0,0.963211,1.0
2,2,1,0.294118,1,14.0,1,0,0,0.070428,152.0,0.849498,0.0
3,3,1,0.411765,1,1.0,0,0.5,1,0.004869,156.0,0.254181,0.0
4,4,1,0.423529,1,15.0,1,0.5,0,0.059150,152.0,0.983278,0.0


In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [6]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame):
        self.id = df['id'].values
        df= df.drop(columns=['id'])

        if 'Response' in df.columns:
            self.response = pd.get_dummies(df['Response']).values
            df= df.drop(columns=['Response'])

        # df = pd.get_dummies(df,columns=['Region_Code','Policy_Sales_Channel'])
        df= df.drop(columns=['Region_Code','Policy_Sales_Channel'])
        self.data = df.astype(float).values
        pass

    def __len__(self):
        return len(self.id)

    def __getitem__(self, idx):
        x=self.data[idx]
        x=torch.tensor(x)
        x=x.to(torch.float32)
        x=x.to(device)

        id = self.id[idx]

        y=self.response[idx]
        y=torch.tensor(y)
        y=y.to(torch.float32)
        y=y.to(device)

        return id, x, y

trainSet= MyDataset(concatDF[:len(trainDF)])
testSet= MyDataset(concatDF[len(trainDF):])
testSet[-1]

(19174663,
 tensor([0.0000, 0.2706, 1.0000, 1.0000, 0.0000, 0.0000, 0.0509, 0.2642],
        device='mps:0'),
 tensor([], device='mps:0'))

In [7]:
splitSet = torch.utils.data.random_split(trainSet,(0.67,0.33))
splitSet

In [8]:
trainLoader=torch.utils.data.DataLoader(splitSet[0],batch_size=1024,sampler=torch.utils.data.RandomSampler(splitSet[0]))
valLoader=torch.utils.data.DataLoader(splitSet[1],batch_size=1024,sampler=torch.utils.data.RandomSampler(splitSet[1]))
testLoader = torch.utils.data.DataLoader(testSet,batch_size=1024)
trainLoader

In [9]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convStack = torch.nn.Sequential(
            torch.nn.Linear(8, 8),
            torch.nn.ReLU(),
            torch.nn.Linear(8, 2),
        )

    def forward(self, x):
        logits = self.convStack(x)
        return logits

model = Net().to(device)
model

Net(
  (convStack): Sequential(
    (0): Linear(in_features=8, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=2, bias=True)
  )
)

In [10]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(),lr=1)
trainHistory=[]
valHistory=[]
lrScheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for phase in range(3):
    bestLoss = float('inf')
    cnt = 0
    bestModel = model

    while(cnt<10):
        model.train()
        trainLoss=0
        for  _, x, y in trainLoader:
            # Compute prediction error
            pred = model(x)
            loss = loss_fn(pred, y)
            trainLoss +=loss.item()

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        valLoss=0
        with torch.no_grad():
            for _, x,y in valLoader:
                pred = model(x)
                loss = loss_fn(pred, y)
                valLoss+=loss.item()

        trainHistory.append(trainLoss)

        valHistory.append(valLoss)

        if bestLoss < valLoss:
            cnt +=1
        else:
            bestLoss = valLoss
            bestModel = model
            cnt =0 

        print(f'{lrScheduler.get_last_lr()} cnt: {cnt} - val loss: {valLoss} - train loss: {trainLoss}')

    model = bestModel
    lrScheduler.step()

plt.plot(trainHistory,label='train')
plt.plot(valHistory, label='val')
plt.legend() 

KeyboardInterrupt: 

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y  in valLoader:
        pred = model(x)

        for idx, i in enumerate(pred):
            if torch.argmax(i)== torch.argmax(y[idx]):
                correct +=1 
            total+=1
correct / total


: 

In [ ]:
def testAI(dataloader, model):
    model.eval()
    out = []
    y=1
    with torch.no_grad():
        for X in dataloader:
            pred = model(X)
            for i in pred:
                out.append([y, torch.argmax(i).item()])
                y+=1

    
    return out

result = testAI(testLoader, bestModel) 
# result = testAI(testLoader[0], model[0])
result

: 

In [ ]:
outDF= pd.DataFrame(result)
outDF= outDF.astype(int)
outDF.columns=['ImageId','Label']
outDF

: 

In [ ]:
outDF.to_csv('result.csv',index=False)

: 